In [2]:
import os
os.listdir('./data')

['product_brand_ner_data.ndjson',
 'final_brand.csv',
 'product_brand_ner_data(with_id).ndjson',
 'brand.ndjson',
 'data_for_crf.csv',
 'data_for_crf.pickle']

In [68]:
brand_df = pd.read_csv('./data/final_brand.csv')

In [74]:
brand_df

,id,name_kor,name_eng,image_url,is_active,used_for_rec,used_for_follow,synonym,model,category
0,1,나이키,Nike,NaN,1,1,1,NaN,NaN,NaN
1,2,스톤아일랜드,Stone Island,NaN,1,1,1,스톤 아일랜드,NaN,NaN
2,3,뉴발란스,NEW BALANCE,NaN,1,1,1,뉴발,NaN,NaN
3,4,아디다스,Adidas,NaN,1,1,1,NaN,이지부스트,NaN
4,5,노스페이스,THE NORTH FACE,NaN,1,1,1,"더노스페이스,노스 페이스, 더 노스페이스, 더 노스 페이스, 노페",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
159,160,BOSE,BOSE,NaN,1,1,1,NaN,NaN,NaN
160,161,하만카돈,Harman Kardon,NaN,1,1,1,하만 카돈,NaN,NaN
161,162,캐논,Canon,NaN,1,1,1,NaN,NaN,NaN
162,163,니콘,Nikon,NaN,1,1,1,NaN,NaN,NaN


In [78]:
cols = ['name_kor', 'name_eng', 'synonym']
for col in cols:
    brand_df[col] = brand_df[col].astype(str)

brand_tmp = brand_df['name_kor'] + ',' + brand_df['name_eng'] + ',' + brand_df['synonym']

In [79]:
brand_tmp

0                                           나이키,Nike,nan
1                            스톤아일랜드,Stone Island,스톤 아일랜드
2                                    뉴발란스,NEW BALANCE,뉴발
3                                        아디다스,Adidas,nan
4      노스페이스,THE NORTH FACE,더노스페이스,노스 페이스, 더 노스페이스, 더...
                             ...                        
159                                        BOSE,BOSE,nan
160                             하만카돈,Harman Kardon,하만 카돈
161                                         캐논,Canon,nan
162                                         니콘,Nikon,nan
163                                   폴라로이드,Polaroid,nan
Length: 164, dtype: object

In [86]:
final_brand = []
for i in brand_tmp:
    final_brand += [s.strip().lower() for s in i.split(',') if s.strip() != 'nan']
final_brand = list(set(final_brand))

In [87]:
len(final_brand)

432

In [88]:
final_brand

['베르사체',
 'bottega veneta',
 '몽블랑',
 '앤더슨 벨',
 '애버크롬비',
 'ikea',
 '꼼데가르송',
 '무신사',
 'marketb',
 'audemars piguet',
 '내셔널지오그래픽',
 '스카티카메론',
 'ami',
 '마시모 두띠',
 '아디다스',
 "levi's",
 '이케아',
 '아더에러',
 '에르메스',
 'gentle monster',
 '태그호이어',
 '할리 데이비슨',
 'a.p.c.',
 'bmw',
 '테일러메이드',
 '슈콤마보니',
 '더 노스페이스',
 'covernat',
 '오니츠카타이거',
 '몽클레어',
 '아크메드라비',
 '포켓몬',
 '랄프로렌',
 '디올',
 '캉골',
 '로렉스',
 'versace',
 '나이키',
 'bang',
 '뱅앤 올룹슨',
 '디스퀘어드',
 '애르매스',
 'epon',
 '폴라로이드',
 '보테가베네타',
 '메종 키츠네',
 'dior',
 '꼼대',
 '디스이즈네버댓',
 'freitag',
 'pandora',
 'mark & lona',
 'lg전자',
 '알파인더스트리',
 'nintendo',
 '포틴',
 'issey miyake',
 'prgr',
 '언더 아머',
 '아식스',
 '우영미',
 'sennheiser',
 'leica',
 '마루망',
 '삼성',
 '로저비비애',
 'pxg',
 'ralph lauren',
 '아배크롬비',
 '레드발렌티노',
 'moncler',
 '꼼대 가르송',
 'comme des garçons',
 'alpha industries',
 'tottenham apparel',
 'patek philippe',
 '혼다',
 '반스',
 '톰브라운',
 'suecomma bonnie',
 'srixon',
 '스톤아일랜드',
 '뉴발',
 '뱅',
 '다이나톤',
 '캐나다구스',
 'acme de la vie',
 '핫토이즈',
 'diesel',
 'onitsuka tiger',


In [4]:
import ndjson
df = None
with open('./data/product_brand_ner_data(with_id).ndjson', 'r', encoding='utf8') as f:
    df = ndjson.load(f)

In [5]:
len(df)

1876305

In [7]:
df[10]

{'brand_id': 1,
 'brands': ['나이키'],
 'nfd_spans': [[0, 6]],
 'spans': [[0, 3]],
 'text': '나이키 네이비 여름 신발 270 265 (택포가격)'}

In [8]:
brand_ids = []
brands = []
texts = []
spans = []
for i in df:
    brand_ids.append(i['brand_id'])
    brands.append(i['brands'])
    texts.append(i['text'])
    spans.append(i['spans'])

In [11]:
def branding(text, spans):
    length = len(text)
    labels = [0] * length
    if len(spans) > 0:
        for span in spans:
            b = span[0]
            e = span[1]
            labels[b:e] = [1] * (e-b)
    return [(text[idx], '1') if value == 1 else (text[idx], '0') for idx, value in enumerate(labels)]

In [12]:
from pycrfsuite_spacing import TemplateGenerator, CharacterFeatureTransformer
from pprint import pprint

templates = TemplateGenerator(
    begin=-2,
    end=2,
    min_range_length=3,
    max_range_length=3
)

to_feature = CharacterFeatureTransformer(templates)

In [13]:
list(to_feature.templates)

[(-2, 0), (-1, 1), (0, 2)]

In [14]:
from pycrfsuite_spacing import sent_to_xy

x, y = sent_to_xy('나이키 네이비 여름 신발 270 265 (택포가격)', to_feature)

In [28]:
x

[['X[0,2]=나이키'],
 ['X[-1,1]=나이키', 'X[0,2]=이키네'],
 ['X[-2,0]=나이키', 'X[-1,1]=이키네', 'X[0,2]=키네이'],
 ['X[-2,0]=이키네', 'X[-1,1]=키네이', 'X[0,2]=네이비'],
 ['X[-2,0]=키네이', 'X[-1,1]=네이비', 'X[0,2]=이비여'],
 ['X[-2,0]=네이비', 'X[-1,1]=이비여', 'X[0,2]=비여름'],
 ['X[-2,0]=이비여', 'X[-1,1]=비여름', 'X[0,2]=여름신'],
 ['X[-2,0]=비여름', 'X[-1,1]=여름신', 'X[0,2]=름신발'],
 ['X[-2,0]=여름신', 'X[-1,1]=름신발', 'X[0,2]=신발2'],
 ['X[-2,0]=름신발', 'X[-1,1]=신발2', 'X[0,2]=발27'],
 ['X[-2,0]=신발2', 'X[-1,1]=발27', 'X[0,2]=270'],
 ['X[-2,0]=발27', 'X[-1,1]=270', 'X[0,2]=702'],
 ['X[-2,0]=270', 'X[-1,1]=702', 'X[0,2]=026'],
 ['X[-2,0]=702', 'X[-1,1]=026', 'X[0,2]=265'],
 ['X[-2,0]=026', 'X[-1,1]=265', 'X[0,2]=65('],
 ['X[-2,0]=265', 'X[-1,1]=65(', 'X[0,2]=5(택'],
 ['X[-2,0]=65(', 'X[-1,1]=5(택', 'X[0,2]=(택포'],
 ['X[-2,0]=5(택', 'X[-1,1]=(택포', 'X[0,2]=택포가'],
 ['X[-2,0]=(택포', 'X[-1,1]=택포가', 'X[0,2]=포가격'],
 ['X[-2,0]=택포가', 'X[-1,1]=포가격', 'X[0,2]=가격)'],
 ['X[-2,0]=포가격', 'X[-1,1]=가격)'],
 ['X[-2,0]=가격)']]

In [18]:
print(y)

['0', '0', '1', '0', '0', '1', '0', '1', '0', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1']


In [33]:
def generate_templates(begin=-2, end=2, min_range_length=3, max_range_length=3):
    templates = []
    for b in range(begin, end):
        for e in range(b, end+1):
            length = (e - b + 1)
            if length > max_range_length or length < min_range_length:
                continue
            if b * e > 0:
                continue
            templates.append((b, e))
    return templates

templates = generate_templates()

In [34]:
def get_features(templates, text, i):
    features = {
        'char_position': i,
        'char': text[i]
    }
    e_max = len(text)
    keys = []
    values = []
    for t in templates:
        b = i + t[0]
        e = i + t[1] + 1
        if b < 0 or e > e_max:
            continue
        keys.append('x[%d : %d]' % (t[0], t[1]))
        values.append([text[b:e]])
    features.update(dict(zip(keys, values)))
    return features

In [37]:
from tqdm import tqdm 

def get_data(brand_ids, texts, spans):
    branding_list = []
    word2features_list = []
    label_list = []
    brand_list = []
    for idx, text in tqdm(enumerate(texts)):
        brand = brand_ids[idx]
        brand_list.append(brand)
        span = spans[idx]
        br = branding(text, span)
        branding_list.append(br)
        w2f = [get_features(templates, text, i) for i in range(len(text))]
        word2features_list.append(w2f)
        label = [label for char, label in br]
        label_list.append(label)
    return {
        'name': texts,
        'branding_list': branding_list, 
        'word2features_list': word2features_list, 
        'label_list': label_list,
        'brand': brand_list
       }

In [38]:
import numpy as np

idx_sample_list = np.random.choice(len(texts), 500000)

data = get_data(np.array(brand_ids)[idx_sample_list].tolist(),
                np.array(texts)[idx_sample_list].tolist(), 
                np.array(spans)[idx_sample_list].tolist())

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
500000it [01:43, 4843.75it/s] 


In [41]:
import pandas as pd
data_as_df = pd.DataFrame.from_dict(data)

In [131]:
data_as_df.head()

,name,branding_list,word2features_list,label_list,brand
0,루이비통 주사위팔찌,"[(루, 1), (이, 1), (비, 1), (통, 1), ( , 0), (주, 0...","[{'char_position': 0, 'char': '루', 'x[0 : 2]':...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]",40
1,컨버스 cx 280,"[(컨, 1), (버, 1), (스, 1), ( , 0), (c, 0), (x, 0...","[{'char_position': 0, 'char': '컨', 'x[0 : 2]':...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]",16
2,백화점 정품 고야드 빅투와르 스페셜 반지갑,"[(백, 0), (화, 0), (점, 0), ( , 0), (정, 0), (품, 0...","[{'char_position': 0, 'char': '백', 'x[0 : 2]':...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, ...",91
3,정품 메종키츠네 반팔티 xl,"[(정, 0), (품, 0), ( , 0), (메, 1), (종, 1), (키, 1...","[{'char_position': 0, 'char': '정', 'x[0 : 2]':...","[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]",11
4,프라이탁 마이애미바이스(택없음),"[(프, 1), (라, 1), (이, 1), (탁, 1), ( , 0), (마, 0...","[{'char_position': 0, 'char': '프', 'x[0 : 2]':...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",32


In [138]:
from sklearn_crfsuite import metrics, CRF
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import pandas as pd

class CrfBrandDetector:
    def __init__(self):
        self.df = None
        self.test_index = None
        self.crf = None
        self.y_pred = None
    
    def train_test_split(self, df, test_size=0.2, random_state=123):
        self.df = df
        x_train, x_test, y_train, y_test = train_test_split(
            self.df['word2features_list'], self.df['label_list'],
            test_size=test_size,
            random_state=random_state
        )
        self.test_index = x_test.index
        return x_train, x_test, y_train, y_test
    
    def fit(self, x_train, y_train):
        self.crf = CRF(
            algorithm='lbfgs',
            c1=0.05,
            c2=0.05,
            max_iterations=100,
            all_possible_states=True
        )
        self.crf.fit(x_train, y_train)
    
    def save_model(self, model_file_name='crf_model.sav'):
        pickle.dump(self.crf, open(model_file_name, 'wb'))
        
    def predict(self, x):
        self.y_pred = self.crf.predict(x)
        product_names = [[diction['char'] for diction in obs] for obs in x]
        pred_tags = []
        pred_brands = []
        for idx, pred in enumerate(self.y_pred):            
            ts = []
            i = 0
            while i < len(pred):
                if pred[i] == '1':
                    if len(ts) > 0 and (i - ts[-1][0]) > 1:
                        ts.append((i, 's')) # start new word
                    else:
                        ts.append((i, 'n')) # not
                i += 1
            pred_tags.append(ts)
        
            predicted = ''
            name = product_names[idx]
            for idx, tag in ts:
                if tag == 'n':
                    predicted += name[idx]
                if tag == 's':
                    predicted += (' ' + name[idx])
            
            pred_brands.append(predicted.split())
        return {
            'product_name': product_names,
            'y_pred': self.y_pred,
            'pred_tags': pred_tags,
            'predicted_brand': pred_brands
            
        }

    
    def report_classification(self, x_test, y_test):
        labels = list(self.crf.classes_)
        labels.remove('0')
        print(metrics.flat_classification_report(
            y_test,
            self.y_pred,
            labels=labels,
            digits=3
        ))

    def evaluate(self, y_test):
        acc = float(list(y_test == self.y_pred).count(True)) / len(self.y_pred)
        return acc

In [139]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

crf_model = CrfBrandDetector()
x_train, x_test, y_train, y_test = crf_model.train_test_split(data_as_df)
crf_model.fit(x_train, y_train)

current_time = datetime.now().strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 17:15:42
End Time = 17:25:45


In [140]:
pred = crf_model.predict(x_test)

In [141]:
pred_as_df = pd.DataFrame.from_dict(pred)

In [146]:
pred_as_df.head()

,product_name,y_pred,pred_tags,predicted_brand
0,"[베, 이, 프, , 케, 이, 스, 티, 파, 이, , 아, 이, 폰, 1, ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(0, n), (1, n), (2, n)]",[베이프]
1,"[[, 2, 5, 0, ~, 2, 8, 0, (, 5, 단, 위, ), ], , ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(15, n), (16, n), (17, n)]",[나이키]
2,"[(, 정, 품, ), , 겐, 조, , 베, 이, 직, , 옐, 로, 우, ...","[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(5, n), (6, n)]",[겐조]
3,"[뉴, 발, 란, 스, , 운, 동, 화, , -, , 2, 4, 0, , ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(0, n), (1, n), (2, n), (3, n)]",[뉴발란스]
4,"[(, x, l, ), , 아, 디, 다, 스, , 후, 드, , 집, 업]","[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]","[(5, n), (6, n), (7, n), (8, n)]",[아디다스]


In [149]:
tmp = pred_as_df[pred_as_df['predicted_brand'].apply(lambda x: len(x) > 1)]

In [158]:
tmp.shape

(5890, 4)

In [160]:
tmp[['product_name', 'predicted_brand']][165:200]

,product_name,predicted_brand
2897,"[s, t, o, n, e, , i, s, l, a, n, d, , (, , ...","[stone, island]"
2909,"[빔, 즈, , b, e, a, m, s, , n, y, , 카, 모, , ...","[빔즈, beams]"
2921,"[코, 스, , c, o, s, , 스, 웨, 터, , m]","[코스, cos]"
2925,"[노, 스, 페, 이, 스, , 7, 0, 0, , 노, 페]","[노스페이스, 노페]"
2934,"[아, 디, 다, 스, , a, d, i, d, a, s, , 흰, 검, , ...","[아디다스, adidas]"
2967,"[c, a, r, h, a, r, t, t, , w, i, p, , 칼, 하, ...","[carhartt, 칼하트]"
2999,"[[, f, r, e, e, ], , 비, 비, 안, , 웨, 스, 트, , ...","[비비안, 웨스트, 우드]"
3011,"[자, 라, , z, a, r, a, , 니, 트]","[자라, zara]"
3054,"[마, 이, 클, , 코, 어, 스, , 가, 방, , 새, 상, 품]","[마이클, 코어스]"
3097,"[y, ', s, , 요, 지, , 야, 마, 모, 토, , 리, 빙, , ...","[요지, 야마모토]"


In [51]:
tmp = y_pred[12]
print(tmp)

['1', '1', '0', '1', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0']


In [113]:
ts = []
i = 0
while i < len(tmp):
    if tmp[i] == '1':
        if len(ts) > 0 and (i - ts[-1][0]) > 1:
            ts.append((i, 's')) # start new word
        else:
            ts.append((i, 'n')) # not
    i += 1

In [114]:
ts

[(0, 'n'), (1, 'n'), (3, 's'), (4, 'n'), (5, 'n'), (6, 'n')]

In [118]:
p_name = pred.loc[12]['product_name']
predicted = ''
for idx, tag in ts:
    if tag == 'n':
        predicted += p_name[idx]
    if tag == 's':
        predicted += (' ' + p_name[idx])

In [121]:
print(p_name)
print(predicted)

자라 zara m사이즈 원피스
자라 zara


In [89]:
notok = []
for i in pred.predicted_brand.unique():
    if i not in final_brand:
        notok.append(i)

In [120]:
pred[pred['predicted_brand'].isin(notok)]

,product_name,predicted_brand
0,베이프 케이스티파이 아이폰11 케이스,베이프
6,루이까스텔 덕다운 패딩/남성105/칠팔구제,루이까스텔
8,데상트 사이즈95,데상트
12,자라 zara m사이즈 원피스,자라zara
20,(당일출고)닐바렛 반팔티셔츠,닐바렛
...,...,...
99983,mcm 송치 토트백 가방 여성가방 가죽,mcm
99988,블레스 bless football bag 풋볼 가방 판매,블레스bless
99991,펜디&벨벳23만원,
99994,더일마 라프시몬스 검정 맨투맨,라프시몬스


In [123]:
pred[pred['predicted_brand'] == '아디다스']

,product_name,predicted_brand
4,(xl) 아디다스 후드 집업,아디다스
5,아디다스 모자,아디다스
21,m 아디다스 반집업 웜업 블랙,아디다스
27,아디다스 배색 맨투맨 xl 후드,아디다스
43,[아디다스] 이지부스트 350 v2 카퍼 (270mm),아디다스
...,...,...
99927,아디다스 이지부스트 275,아디다스
99938,95) 아디다스 양면 패딩 조끼 오리털,아디다스
99964,260) 아디다스 코트밴티지 운동화,아디다스
99972,아디다스 농구화 뱨송비 포함,아디다스


In [48]:
pred[:20]

,product_name,predicted_brand
0,베이프 케이스티파이 아이폰11 케이스,베이프
1,[250~280(5단위)] 나이키 에어포스 로우 검흰,나이키
2,(정품) 겐조 베이직 옐로우 그린 반팔 셔츠,겐조
3,뉴발란스 운동화 - 240 ( 새 제품 ) 팝니다!!,뉴발란스
4,(xl) 아디다스 후드 집업,아디다스
5,아디다스 모자,아디다스
6,루이까스텔 덕다운 패딩/남성105/칠팔구제,루이까스텔
7,[m] 무스너클 블랙 골드 폭스퍼 스틸링 여성 패딩점퍼,무스너클
8,데상트 사이즈95,데상트
9,[50] 우영미 19fw 네이비 보더 스트라이프 니트 베스트,우영미


In [47]:
crf_model.report_classification(x_test, y_test)

              precision    recall  f1-score   support

           1      0.994     0.996     0.995    382748

   micro avg      0.994     0.996     0.995    382748
   macro avg      0.994     0.996     0.995    382748
weighted avg      0.994     0.996     0.995    382748



In [192]:
import numpy as np
acc = float(list(np.array(y_test) == np.array(y_pred)).count(True))/len(y_pred)

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [196]:
round(acc, 6)

0.99328